In [71]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import metpy
import metpy.calc
from metpy.units import units
from metpy.calc import equivalent_potential_temperature
from metpy.calc import dewpoint_from_relative_humidity
from metpy.calc import dewpoint_from_specific_humidity
from datetime import datetime
import pandas as pd

In [96]:
def open_files(variable,lat_start,lat_end,lon_start,lon_end):
    list = ["202201-202201","202202-202202","202101-202101","202102-202102","202103-202103","202104-202104","202001-202001","202002-202002",
           "202003-202003","202004-202004","201901-201901","201902-201902","201903-201903","201904-201904","201801-201801","201802-201802",
           "201803-201803","201804-201804","201701-201701","201702-201702","201703-201703","201704-201704","201601-201601","201602-201602",
           "201603-201603","201604-201604","201501-201501","201502-201502","201503-201503","201504-201504","201401-201401","201402-201402",
           "201403-201403","201404-201404","201301-201301","201302-201302","201303-201303","201304-201304","201201-201201","201202-201202",
           "201203-201203","201204-201204"
            ] 
    fp = "/g/data/yb19/australian-climate-service/release/ACS-BARRA2/output/AUS-11/BOM/ECMWF-ERA5/historical/hres/BOM-BARRA-R2/v1/1hr/"+variable+"/"
    data = []
    for months in list:
        # to get specific hour, edit: u.ua850.isel(time=(barra2_202201.time.dt.hour == hour)
        var = xr.open_dataset(fp+variable+"_AUS-11_ECMWF-ERA5_historical_hres_BOM-BARRA-R2_v1_1hr_"+months+".nc", 
                              engine="h5netcdf",chunks={'time':-1}) #'auto'
        domain = var[variable].sel(lat=slice(lat_start,lat_end),lon=slice(lon_start,lon_end))
        var_mean = domain.mean(dim=["lat","lon"])
        data.append(var_mean)
    return data

In [97]:
%%time
# variables for entire domain #towns:-20.768799,-18.0708,145.12054,147.9812
tas = open_files("tasmean",-20.768799,-18.0708,145.12054,147.9812)#-22,-14,143,152) # hourly mean near surface temperature
ps = open_files("ps",-20.768799,-18.0708,145.12054,147.9812)#-22,-14,143,152)       # hourly surface pressure
huss = open_files("huss",-20.768799,-18.0708,145.12054,147.9812)#-22,-14,143,152)   # hourly near surface specific humidity
hurs = open_files("hurs",-20.768799,-18.0708,145.12054,147.9812)#-22,-14,143,152)   # hourly near surface relative humidity

CPU times: user 12.5 s, sys: 2.59 s, total: 15.1 s
Wall time: 30.8 s


In [98]:
%%time
# concat by time
tas_concat = xr.concat(tas,"time")
ps_concat = xr.concat(ps,"time")
huss_concat = xr.concat(huss,"time")
hurs_concat = xr.concat(hurs,"time")

CPU times: user 302 ms, sys: 27 ms, total: 329 ms
Wall time: 348 ms


In [100]:
%%time
temp_celsius = (tas_concat - 273.15) * units("degC")
# degc_tas = (temp_celsius * units("degC")).compute()

CPU times: user 12.9 ms, sys: 0 ns, total: 12.9 ms
Wall time: 11.5 ms


In [101]:
%%time
Tps_hPa = (ps_concat/100) * units("hPa")
# hPa_ps = (Tps_hPa * units("hPa")).compute()

CPU times: user 10.7 ms, sys: 1.25 ms, total: 12 ms
Wall time: 11.9 ms


In [102]:
%%time
# Thuss = (huss_concat * units("g/kg")).compute() 
Thuss_gkg = (huss_concat * 100) * units("g/kg")                        # convert from kg/kg
# gkg_huss = (Thuss_gkg * units("g/kg")).compute()

CPU times: user 4.38 ms, sys: 0 ns, total: 4.38 ms
Wall time: 4.3 ms


In [103]:
%%time
hurs_ratio = (hurs_concat/100) * units("percent")                          # convert into ratio between 0 < rh <= 1
# ratio_hurs = (hurs_ratio * units("percent")).compute()

CPU times: user 17.7 ms, sys: 0 ns, total: 17.7 ms
Wall time: 16.9 ms


In [130]:
%%time
# calculate dewpoint
dewpointRH = metpy.calc.dewpoint_from_relative_humidity(tas_concat*units.degC, (hurs_concat/100) * units("percent"))  # temperature in deg C, relative humidity in ratio between 0< rh <=1
# dewpointSH = metpy.calc.dewpoint_from_specific_humidity(hPa_ps, degc_tas, gkg_huss)  # pressure in hPa, temperature in deg C, specific humidity in g/kg

CPU times: user 5min 4s, sys: 21.2 s, total: 5min 26s
Wall time: 5min 35s


In [133]:
dewpointRH

<xarray.DataArray 'truediv-ada23467d84f051ef8f05acfde2fbda3' (time: 60576)>
<Quantity(dask.array<truediv, shape=(60576,), dtype=float32, chunksize=(1488,), chunktype=numpy.ndarray>, 'degree_Celsius')>
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 ... 2022-02-28T23:30:00
    height   float64 1.5

In [78]:
# calculate theta e
theta_e_rh = metpy.calc.equivalent_potential_temperature(hPa_ps,degc_tas, degc_tas)
# theta_e_sh = metpy.calc.equivalent_potential_temperature(hPa_ps,degc_tas, degc_tas)

In [ ]:
time=xr.DataArray(
    dewpointRH.time,
    dims = ['time','lat','lon'], 
    coords = {'time': dewpointRH.time},
    attrs = {'time_period':'JFMA 2012-2022'},
)
dewpointRH_array = dewpointRH.values
dewpointSH_array = dewpointSH.values
theta_e_rh_array = theta_e_rh.values
theta_e_sh_array = theta_e_sh.values
da1 = xr.DataArray(dewpointRH_array, dims=('time','lat','lon'), coords={'time': dewpointRH.time,'lat':dewpointRH.lat,'lon':dewpointRH.lon}, name='dewpointRH')
da2 = xr.DataArray(dewpointSH_array, dims=('time','lat','lon'), coords={'time': dewpointRH.time,'lat':dewpointRH.lat,'lon':dewpointRH.lon}, name='dewpointSH')
da3 = xr.DataArray(theta_e_rh_array, dims=('time','lat','lon'), coords={'time': dewpointRH.time,'lat':dewpointRH.lat,'lon':dewpointRH.lon}, name='theta_e_rh')
da4 = xr.DataArray(theta_e_sh_array, dims=('time','lat','lon'), coords={'time': dewpointRH.time,'lat':dewpointRH.lat,'lon':dewpointRH.lon}, name='theta_e_sh')